In [132]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
import pandas as pd
import numpy as np
import statistics as st
model = RandomForestClassifier(n_estimators = 1000, random_state = 2020, criterion = 'entropy', max_depth = 5)

In [139]:
df = pd.read_csv("train.csv")
X = df[[column for column in df.columns if column != 'target']]
numerical_cols = [col for col in df.columns if df[col].dtype in ['int64', 'float64']]
categorical_cols = [col for col in df.columns if df[col].dtype == 'object']
#features = ["Policy Start Date", "Policy End Date", "Gender", "Age", "First Transaction Date", "No_Pol"]
y = df.target

In [145]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy = 'constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'most_frequent')), ('onehot', OneHotEncoder(handle_unknown = 'ignore'))])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[('num', numerical_transformer, numerical_cols), ('cat', categorical_transformer, categorical_cols)])

In [146]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 2020, train_size = 0.8, stratify = y)
# xg_class = xgb.XGBClassifier(objective = 'reg:logistic', max_depth = 10, n_estimators = 1000, learning_rate = 0.9, alpha = 10)

# xg_class.fit(X_train, y_train)

# y_tr = xg_class.predict(X_train)
# y_preds = xg_class.predict(X_valid)

In [147]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', model)])

my_pipeline.fit(X_train, y_train)

y_tr = my_pipeline.predict(X_train)
preds = my_pipeline.predict(X_valid)

ValueError: A given column is not a column of the dataframe

In [138]:
# X = df[features]
# X['Policy Start Date'] = X['Policy Start Date'].map(lambda p: p.split('-')[0])
# X['Policy End Date'] = X['Policy End Date'].map(lambda x: x.split('-')[0])
# X['First Transaction Date'] = X['First Transaction Date'].map(lambda x: x.split('-')[0])

# X.head()

In [131]:
# X["Policy Start Date"] = X["Policy Start Date"].apply(lambda value: int(value) - int(X['Policy Start Date'].min()))
# X["Policy End Date"] = X["Policy End Date"].apply(lambda value: int(value) - int(X['Policy End Date'].min()))
# X["First Transaction Date"] = X["First Transaction Date"].apply(lambda value: int(value) - int(X['First Transaction Date'].min()))

In [126]:
# X['Gender'] = X['Gender'].apply(lambda x: 1 if x == 'Male' else 0)
# X = X.drop(X[(X.Gender > 75) & (X.Gender < 15)].index)
# X.dtypes

<ipython-input-126-8e970f01cbad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Gender'] = X['Gender'].apply(lambda x: 1 if x == 'Male' else 0)


Policy Start Date         int64
Policy End Date           int64
Gender                    int64
Age                       int64
First Transaction Date    int64
No_Pol                    int64
dtype: object

In [127]:
# min_policy_start = X['Policy Start Date'].min()
# min_policy_end = X['Policy End Date'].min()
# min_transaction = X['First Transaction Date'].min()

# X['Policy Start Date'].app - min_policy_start
# for uplet in zip(['Policy Start Date', 'Policy End Date', 'First Transaction Date'], [min_policy_start, min_policy_end, min_transaction]):
#     X[uplet[0]]

In [129]:
data_matrix = xgb.DMatrix(data = X, label = df['target'], enable_categorical = True)


In [130]:
print("Classification Report : \n", classification_report(y_valid, y_preds))



print("For the training set, \t", f1_score(y_train, y_tr))

print("For the Validation set \t", f1_score(y_valid, y_preds))

Classification Report : 
               precision    recall  f1-score   support

           0       0.89      1.00      0.94      2125
           1       0.83      0.07      0.13       291

    accuracy                           0.89      2416
   macro avg       0.86      0.53      0.53      2416
weighted avg       0.88      0.89      0.84      2416

For the training set, 	 0.08555286521388217
For the Validation set 	 0.126984126984127
